In [ ]:
!conda install -c conda-forge xesmf --yes
!pip install zarr

In [2]:

# conda install -c conda-forge xesmf
# pip install cftime

import dask
# from dask_gateway import Gateway

import s3fs
import gcsfs
import xarray as xr
import pandas as pd
import numpy as np
import xesmf as xe
import matplotlib.pyplot as plt
import zarr
import cftime
import tqdm
import datetime

xr.set_options(display_style='html')
%matplotlib inline
%config InlineBackend.figure_format = 'retina' 
plt.rcParams['figure.figsize'] = 12, 6

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

In [3]:
### data loading function
fs = s3fs.S3FileSystem(anon=True, default_fill_cache=False)
gcs = gcsfs.GCSFileSystem(token='anon')

In [4]:
s3_url = 's3://cmip6-pds/cmip6-zarr-consolidated-stores.csv'
gg_url = 'https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv'

In [5]:
### read data index
df = pd.read_csv(gg_url)
df.tail(5)

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
493419,CMIP,AS-RCEC,TaiESM1,historical,r2i1p1f1,Amon,va,gn,gs://cmip6/CMIP6/CMIP/AS-RCEC/TaiESM1/historic...,NaN,20210416
493420,CMIP,AS-RCEC,TaiESM1,historical,r2i1p1f1,Amon,zg,gn,gs://cmip6/CMIP6/CMIP/AS-RCEC/TaiESM1/historic...,NaN,20210416
493421,CMIP,AS-RCEC,TaiESM1,historical,r2i1p1f1,Amon,ua,gn,gs://cmip6/CMIP6/CMIP/AS-RCEC/TaiESM1/historic...,NaN,20210416
493422,CMIP,AS-RCEC,TaiESM1,historical,r2i1p1f1,Amon,ta,gn,gs://cmip6/CMIP6/CMIP/AS-RCEC/TaiESM1/historic...,NaN,20210416
493423,CMIP,AS-RCEC,TaiESM1,historical,r2i1p1f1,Amon,wap,gn,gs://cmip6/CMIP6/CMIP/AS-RCEC/TaiESM1/historic...,NaN,20210416


In [6]:
### find the model that has both historical and scenario records 
df_his_585 = df.query("variable_id == 'pr' & experiment_id == 'historical' & table_id == 'day' & member_id == 'r1i1p1f1' & activity_id=='CMIP'")
df_ssp_585 = df.query("variable_id == 'pr' & experiment_id == 'ssp585'     & table_id == 'day' & member_id == 'r1i1p1f1'")
df_merged_ssp_585  = pd.merge(df_his_585, df_ssp_585, on =['source_id','member_id','table_id','variable_id','grid_label'], how='inner')

df_his_245 = df.query("variable_id == 'pr' & experiment_id == 'historical' & table_id == 'day' & member_id == 'r1i1p1f1' & activity_id=='CMIP'")
df_ssp_245 = df.query("variable_id == 'pr' & experiment_id == 'ssp245'     & table_id == 'day' & member_id == 'r1i1p1f1'")
df_merged_ssp_245  = pd.merge(df_his_245, df_ssp_245, on =['source_id','member_id','table_id','variable_id','grid_label'], how='inner')

df_his_126 = df.query("variable_id == 'pr' & experiment_id == 'historical' & table_id == 'day' & member_id == 'r1i1p1f1' & activity_id=='CMIP'")
df_ssp_126 = df.query("variable_id == 'pr' & experiment_id == 'ssp126'     & table_id == 'day' & member_id == 'r1i1p1f1'")
df_merged_ssp_126  = pd.merge(df_his_126, df_ssp_126, on =['source_id','member_id','table_id','variable_id','grid_label'], how='inner')

### get intersection
historical_url = list(set(df_merged_ssp_245['zstore_x']).intersection(set(df_merged_ssp_126['zstore_x'])).intersection(set(df_merged_ssp_585['zstore_x'])))

### get ssp 585
ssp245_url = df_merged_ssp_245[df_merged_ssp_245['zstore_x'].isin(historical_url)]['zstore_y'].tolist()

In [7]:
print('number of models: ', len(ssp245_url))
ssp245_url.sort()
ssp245_url[:5]

number of models:  24


['gs://cmip6/CMIP6/ScenarioMIP/BCC/BCC-CSM2-MR/ssp245/r1i1p1f1/day/pr/gn/v20190318/',
 'gs://cmip6/CMIP6/ScenarioMIP/CCCR-IITM/IITM-ESM/ssp245/r1i1p1f1/day/pr/gn/v20200915/',
 'gs://cmip6/CMIP6/ScenarioMIP/CCCma/CanESM5/ssp245/r1i1p1f1/day/pr/gn/v20190429/',
 'gs://cmip6/CMIP6/ScenarioMIP/CMCC/CMCC-CM2-SR5/ssp245/r1i1p1f1/day/pr/gn/v20200617/',
 'gs://cmip6/CMIP6/ScenarioMIP/CMCC/CMCC-ESM2/ssp245/r1i1p1f1/day/pr/gn/v20210129/']

### read and store SSP245 data

In [13]:
# dictonary to collect data
ds_245_dictonary = {}
for i in tqdm.tqdm(range(len(ssp245_url))):
    zstore = ssp245_url[i]

   ### load data, use_cftime=True helps line up with the date format
    try:
        mapper = fs.get_mapper(zstore.replace("gs://cmip6", "s3://cmip6-pds"))
        ds_245 = xr.open_zarr(mapper, consolidated=True, decode_times=True, use_cftime=True)
    except KeyError:
        mapper = gcs.get_mapper(zstore.replace("s3://cmip6-pds", "gs://cmip6"))
        ds_245 = xr.open_zarr(mapper, consolidated=True, decode_times=True, use_cftime=True)

    ### keep data within 1950-2014
    ds_245 = ds_245.sel(time = slice('2015', '2100'))
    min_year = ds_245.time.values.min().year
    max_year = ds_245.time.values.max().year
    list_of_year = range(min_year, max_year + 1)
    if min(list_of_year) != 2015:
        print(str(i) + ': min year of the model is not 2015! It is: ', str(min(list_of_year)))
    if max(list_of_year) != 2100:
        print(str(i) + ': max year of the model is not 2100! It is: ', str(max(list_of_year)))

    ### drop model with 360 days
    for y in list_of_year:
        days = len(ds_245.sel(time = str(y)).time.values)
        if days == 360:
            print('model ' + str(i) + ' has 360 days in a year!!')
            break
            
    else:
        ### Drop leap day
        ds_245 = ds_245.where((ds_245['time.month'] != 2) | (ds_245['time.day'] != 29), drop=True)
        days = len(ds_245.sel(time = str(y)).time.values)
        if days != 365:
            print('model ' + str(i) + ' year ' + str(y) + ' does not have 365 days! It has ' + str(days) + ' days!!' )
        ds_245_dictonary[i] = ds_245

  4%|▍         | 1/24 [00:01<00:37,  1.63s/it]

1: max year of the model is not 2100! It is:  2099


 92%|█████████▏| 22/24 [00:33<00:02,  1.43s/it]

model 21 has 360 days in a year!!


100%|██████████| 24/24 [00:36<00:00,  1.52s/it]


### Check if any model is missing lat/lon bounds

In [14]:
for i in tqdm.tqdm(ds_245_dictonary.keys()):
    try:
        ds_245_dictonary[i].lat_bnds
    except AttributeError:
        print('ssp245 model ' + str(i) + ' does not have lat bounds!!')
    try:
        ds_245_dictonary[i].lon_bnds
    except AttributeError:
        print('ssp245 model ' + str(i) + ' does not have lon bounds!!')

100%|██████████| 23/23 [00:00<00:00, 2792.16it/s]

ssp245 model 13 does not have lat bounds!!
ssp245 model 13 does not have lon bounds!!


### Regrid + cftime to numpy.dateframe

In [15]:
ds_out = xr.Dataset({'lat': (['lat'], np.arange(-89.5, 90.0, 1.0)),
                     'lat_bnds' : (['lat', 'bnds'], np.array([[x, x + 1] for x in range(-90,90)])),
                     'lon': (['lon'], np.arange(0.5, 360.0, 1.0)),
                     'lon_bnds' : (['lon', 'bnds'], np.array([[x, x + 1] for x in range(0,360)])),
                    })

for i in tqdm.tqdm(ds_245_dictonary.keys()):
    regridder = xe.Regridder(ds_245_dictonary[i], ds_out, 'nearest_s2d', reuse_weights=False)
    ds_245_dictonary[i] = regridder(ds_245_dictonary[i]) 
    regridder._grid_in = None
    regridder._grid_out = None
    ds_245_dictonary[i]['time'] = ds_245_dictonary[i].indexes['time'].to_datetimeindex().normalize()

print('SSP245 - Done!')

  0%|          | 0/23 [00:00<?, ?it/s]

Overwrite existing file: nearest_s2d_160x320_180x360.nc 
 You can set reuse_weights=True to save computing time.
using dimensions ('lat', 'lon') from data variable pr as the horizontal dimensions for this dataset.


  4%|▍         | 1/23 [00:00<00:11,  1.90it/s]

Overwrite existing file: nearest_s2d_94x192_180x360.nc 
 You can set reuse_weights=True to save computing time.
using dimensions ('lat', 'lon') from data variable pr as the horizontal dimensions for this dataset.


  9%|▊         | 2/23 [00:01<00:10,  2.01it/s]

...

Overwrite existing file: nearest_s2d_180x288_180x360.nc 
 You can set reuse_weights=True to save computing time.
using dimensions ('lat', 'lon') from data variable pr as the horizontal dimensions for this dataset.


 96%|█████████▌| 22/23 [00:11<00:00,  1.95it/s]

Overwrite existing file: nearest_s2d_96x192_180x360.nc 
 You can set reuse_weights=True to save computing time.
using dimensions ('lat', 'lon') from data variable pr as the horizontal dimensions for this dataset.


100%|██████████| 23/23 [00:11<00:00,  1.96it/s]

SSP245 - Done!


### Mean & Std Calculation for SSP245

In [16]:
# model 3 only has data till 2099, so we fill in 2100 data with nan
ds_245_3 = xr.concat([ds_245_dictonary[1], ds_245_dictonary[0].sel(time = slice('2100-01-01','2100-12-31'))], dim = 'time')
ds_245_3['pr'] = xr.where(ds_245_3.time >= np.datetime64("2100-01-01T12:00:00"), np.nan, ds_245_3.pr)

ds_245_dictonary[1] = ds_245_3

In [17]:
ds_245_dictonary

{0: <xarray.Dataset>
 Dimensions:    (bnds: 2, lat: 180, lon: 360, time: 31390)
 Coordinates:
   * time       (time) datetime64[ns] 2015-01-01 2015-01-02 ... 2100-12-31
     time_bnds  (time, bnds) object dask.array<chunksize=(15695, 2), meta=np.ndarray>
   * lon        (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
   * lat        (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
 Dimensions without coordinates: bnds
 Data variables:
     pr         (time, lat, lon) float64 dask.array<chunksize=(600, 180, 360), meta=np.ndarray>
 Attributes:
     regrid_method:  nearest_s2d, 1: <xarray.Dataset>
 Dimensions:    (bnds: 2, lat: 180, lon: 360, time: 31390)
 Coordinates:
   * time       (time) datetime64[ns] 2015-01-01 2015-01-02 ... 2100-12-31
     time_bnds  (time, bnds) object dask.array<chunksize=(15512, 2), meta=np.ndarray>
   * lon        (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
   * lat        (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 8

In [18]:
# create new dimension "model"
ds_245_temp = ds_245_dictonary[0].expand_dims({'model':list(ds_245_dictonary.keys())})
for i in tqdm.tqdm(ds_245_dictonary.keys()):
    ds_245_temp['pr'] = xr.where(ds_245_temp.model == i, ds_245_dictonary[i].pr, ds_245_temp.pr)

100%|██████████| 23/23 [00:00<00:00, 41.24it/s]


In [20]:
ds_245_output = ds_245_temp.mean('model').rename_vars({'pr':'pr_mean'})
ds_245_output['std'] = ds_245_temp.std('model').pr
ds_245_output

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 180, lon: 360, time: 31390)
Coordinates:
  * time       (time) datetime64[ns] 2015-01-01 2015-01-02 ... 2100-12-31
    time_bnds  (time, bnds) object dask.array<chunksize=(15695, 2), meta=np.ndarray>
  * lon        (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * lat        (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
Dimensions without coordinates: bnds
Data variables:
    pr_mean    (time, lat, lon) float64 dask.array<chunksize=(127, 180, 360), meta=np.ndarray>
    std        (time, lat, lon) float64 dask.array<chunksize=(127, 180, 360), meta=np.ndarray>

In [ ]:
ds_245_output.to_netcdf('ssp245_daily_pr.nc')

Unclosed connection
client_connection: Connection<ConnectionKey(host='cmip6-pds.s3.amazonaws.com', port=443, is_ssl=True, ssl=None, proxy=None, proxy_auth=None, proxy_headers_hash=None)>
Unclosed connection
client_connection: Connection<ConnectionKey(host='cmip6-pds.s3.amazonaws.com', port=443, is_ssl=True, ssl=None, proxy=None, proxy_auth=None, proxy_headers_hash=None)>
Unclosed connection
client_connection: Connection<ConnectionKey(host='cmip6-pds.s3.amazonaws.com', port=443, is_ssl=True, ssl=None, proxy=None, proxy_auth=None, proxy_headers_hash=None)>
Unclosed connection
client_connection: Connection<ConnectionKey(host='cmip6-pds.s3.amazonaws.com', port=443, is_ssl=True, ssl=None, proxy=None, proxy_auth=None, proxy_headers_hash=None)>
Unclosed connection
client_connection: Connection<ConnectionKey(host='cmip6-pds.s3.amazonaws.com', port=443, is_ssl=True, ssl=None, proxy=None, proxy_auth=None, proxy_headers_hash=None)>
Unclosed connection
client_connection: Connection<ConnectionKey(h

In [48]:
!aws s3 cp ssp245_daily.nc s3://aer-astd-mcclim/ssp245/ssp245_daily.nc

upload: ./ssp245_daily.nc to s3://aer-astd-mcclim/ssp245/ssp245_daily.nc


### SSP245 Monthly

In [15]:
ds_245_output['CDD'] = xr.where(((ds_245_output.tas_mean - 273.15) * 9/5 + 32) > 65, ((ds_245_output.tas_mean - 273.15) * 9/5 + 32) - 65, 0)
ds_245_output['HDD'] = xr.where(((ds_245_output.tas_mean - 273.15) * 9/5 + 32) < 65, 65 - ((ds_245_output.tas_mean - 273.15) * 9/5 + 32), 0)
ds_245_output_monthly = ds_245_output.resample(time = 'M').sum(dim = 'time')[['CDD', 'HDD']]
ds_245_output_monthly

<xarray.Dataset>
Dimensions:  (lat: 180, lon: 360, time: 540)
Coordinates:
  * time     (time) datetime64[ns] 2056-01-31 2056-02-29 ... 2100-12-31
    height   float64 2.0
  * lon      (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 355.5 356.5 357.5 358.5 359.5
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
Data variables:
    CDD      (time, lat, lon) float64 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    HDD      (time, lat, lon) float64 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>

### Output netcdf file

In [16]:
ds_245_output_monthly.to_netcdf('ssp245_monthly.nc')

In [47]:
!aws s3 cp ssp245_monthly.nc s3://aer-astd-mcclim/ssp245/ssp245_monthly.nc

upload: ./ssp245_monthly.nc to s3://aer-astd-mcclim/ssp245/ssp245_monthly.nc


### Verify the result

In [24]:
t = 0
n = 0
for i in range(0, 25):
    if i not in [22, 3]:
        t += ds_245_dictonary[i].sel(time = '2100-11-22', lon = 215.5, lat = -4.5).tas.values
        n += 1
t/n

301.41091388204825

In [20]:
ds_daily = xr.open_dataset("ssp245_daily.nc")
ds_monthly = xr.open_dataset("ssp245_monthly.nc")

In [25]:
ds_daily.sel(time = '2100-11-22', lon = 215.5, lat = -4.5).tas_mean.values

array(301.41091388)

In [42]:
ds = ds_daily.sel(time = '2100-01', lon = 0.5, lat = 0.5)

In [43]:
ds['HDD'] = xr.where(((ds.tas_mean - 273.15) * 9/5 + 32) < 65, 65 - ((ds.tas_mean - 273.15) * 9/5 + 32), 0)
ds['CDD'] = xr.where(((ds.tas_mean - 273.15) * 9/5 + 32) > 65, ((ds.tas_mean - 273.15) * 9/5 + 32) - 65, 0)

In [44]:
print('CDD: ', ds.CDD.values.sum())
print('HDD: ', ds.HDD.values.sum())

CDD:  616.4485880246383
HDD:  0.0


In [45]:
d = ds_monthly.sel(time = '2100-01', lon = 0.5, lat = 0.5)
print('CDD: ', d.CDD.values)
print('HDD: ', d.HDD.values)

CDD:  [616.44858802]
HDD:  [0.]


In [52]:
ds_daily_245 = xr.open_dataset("ssp245_daily.nc")
ds_daily_585 = xr.open_dataset("ssp585_daily.nc")

In [67]:
ds_daily_245.sel(time = '2050-11-22', lon = 125.5, lat = -64.5).tas_mean.values

array(270.06465022)

In [68]:
ds_daily_585.sel(time = '2050-11-22', lon = 125.5, lat = -64.5).tas_mean.values

array(271.01663589)